In [1]:
import torch
import numpy as np
import copy
from torch_sparse.tensor import SparseTensor

from sketch import CountSketch, TensorSketch
from sketch import Backends, OutputTypes, SketchConfig, DEFAULT_COUNT_SKETCH_CONFIG, DEFAULT_TENSOR_SKETCH_CONFIG
from initialize_sketch import initialize_single_layer_sketch_modules

## Validate and Benchmark Count-Sketch

In [2]:
modes = ['dense_CPU', 'dense_GPU', 'sparse_CPU', 'sparse_GPU']
backends = [Backends.PYTORCH, Backends.PYTORCH_SPARSE, Backends.PYG_SPARSE, Backends.PYG_SCATTER]

in_dim = 2048
out_dim = 128
feature_dim = 128
repeats = 128
log_interval = 32
ntrials = 3
max_degree = 3
avg_node_degree = 64

row = torch.arange(in_dim, dtype=torch.int64).repeat_interleave(avg_node_degree)
u0_col = torch.from_numpy(np.concatenate([np.random.choice(feature_dim, avg_node_degree, replace=False) for _ in range(in_dim)], axis=0, dtype=np.int64))
u0_val = torch.rand(avg_node_degree*in_dim, dtype=torch.float32)
u0_s = SparseTensor(row=row, col=u0_col, value=u0_val, sparse_sizes=(in_dim, feature_dim), is_sorted=False)
v0_col = torch.from_numpy(np.concatenate([np.random.choice(feature_dim, avg_node_degree, replace=False) for _ in range(in_dim)], axis=0, dtype=np.int64))
v0_val = torch.rand(avg_node_degree*in_dim, dtype=torch.float32)
v0_s = SparseTensor(row=row, col=v0_col, value=v0_val, sparse_sizes=(in_dim, feature_dim), is_sorted=False)
u0_d = u0_s.to_dense()
v0_d = v0_s.to_dense()

In [3]:
def validate_count_sketch(x0, config):
    device = x0.device if isinstance(x0, torch.Tensor) else x0.device()
    rel_errors = np.zeros((repeats//log_interval, ntrials))
    density = 0
    for t in range(ntrials):
        if isinstance(x0, torch.Tensor):
            usx0 = torch.zeros_like(x0).to(device)
        else:
            usx0 = torch.zeros((in_dim, feature_dim), dtype=torch.float32).to(device)
        for r in range(1, repeats+1):
            count_sketch = CountSketch(in_dim, out_dim, config).to(device)
            _csx0 = count_sketch(x0)
            csx0 = _csx0 if isinstance(_csx0, torch.Tensor) else _csx0.to_dense()
            density += torch.count_nonzero(csx0)/out_dim/feature_dim
            _usx0 = count_sketch.unsketch_mat(_csx0)
            usx0 += _usx0 if isinstance(_usx0, torch.Tensor) else _usx0.to_dense()
            x0_norm = torch.norm(x0 if isinstance(x0, torch.Tensor) else x0.to_dense(), dim=0)
            if r % log_interval == 0:
                rel_errors[r//log_interval-1, t] = (torch.sum(torch.abs(x0_norm-torch.norm(usx0/r, dim=0)))/
                                                    torch.sum(x0_norm)).cpu().numpy()
    for i in range(repeats//log_interval):
        avg_rel_error = np.mean(rel_errors, axis=1)
        std_rel_error = np.std(rel_errors, axis=1)
        print(f"# of repeats: {log_interval*(i+1):03d},\
              relative error: {avg_rel_error[i]*100:.2f}+/-{std_rel_error[i]*100:.2f}%")
    print(f"output density: {density/ntrials/repeats*100:.2f}%")

print("ConutSketch:")
for mode in modes:
    for backend in backends:
        config = copy.copy(DEFAULT_COUNT_SKETCH_CONFIG)
        setattr(config, mode, backend)
        print(f"Mode: {mode}, Backend: {backend}")
        if mode.startswith('dense'):
            x0 = u0_d
        else:
            x0 = u0_s
        if mode.endswith('CPU'):
            x0 = x0.cpu()
        else:
            x0 = x0.cuda()
        try:
            validate_count_sketch(x0, config)
        except NotImplementedError as e:
            print('Not Implemented')

ConutSketch:
Mode: dense_CPU, Backend: Backends.PYTORCH
# of repeats: 032,              relative error: 23.48+/-0.26%
# of repeats: 064,              relative error: 12.35+/-0.49%
# of repeats: 096,              relative error: 8.45+/-0.49%
# of repeats: 128,              relative error: 6.41+/-0.59%
output density: 99.97%
Mode: dense_CPU, Backend: Backends.PYTORCH_SPARSE
# of repeats: 032,              relative error: 22.17+/-0.85%
# of repeats: 064,              relative error: 11.32+/-0.54%
# of repeats: 096,              relative error: 7.80+/-0.35%
# of repeats: 128,              relative error: 6.06+/-0.39%
output density: 99.97%
Mode: dense_CPU, Backend: Backends.PYG_SPARSE
# of repeats: 032,              relative error: 22.75+/-1.05%
# of repeats: 064,              relative error: 11.83+/-0.81%
# of repeats: 096,              relative error: 7.96+/-0.70%
# of repeats: 128,              relative error: 5.97+/-0.76%
output density: 99.97%
Mode: dense_CPU, Backend: Backends.PYG_SC

In [4]:
print("ConutSketch:")
for mode in modes:
    for backend in backends:
        config = copy.copy(DEFAULT_COUNT_SKETCH_CONFIG)
        setattr(config, mode, backend)
        print(f"Mode: {mode}, Backend: {backend}")
        if mode.startswith('dense'):
            x0 = u0_d
        else:
            x0 = u0_s
        if mode.endswith('CPU'):
            x0 = x0.cpu()
            count_sketch = CountSketch(in_dim, out_dim, config).cpu()
        else:
            x0 = x0.cuda()
            count_sketch = CountSketch(in_dim, out_dim, config).cuda()
        try:
            count_sketch(x0)
            print("Unit time: ", end="")
            %timeit count_sketch(x0)
        except NotImplementedError as e:
            print('Not Implemented')

ConutSketch:
Mode: dense_CPU, Backend: Backends.PYTORCH
Unit time: 1.61 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 1.01 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SPARSE
Unit time: The slowest run took 8.77 times longer than the fastest. This could mean that an intermediate result is being cached.
2.32 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SCATTER
Unit time: 1.28 ms ± 87.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH
Unit time: 43.1 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 507 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SPARSE
Unit time: 50.9 µs ± 4.89 µs per loop (mean

In [5]:
print("ConutSketch:")
for backend in backends:
    config = copy.copy(DEFAULT_COUNT_SKETCH_CONFIG)
    setattr(config, 'dense_GPU', backend)
    print(f"Backend: {backend}")
    try:
        torch.cuda.empty_cache()
        count_sketch = CountSketch(in_dim, out_dim, config).cuda()
        mem_before = torch.cuda.memory_allocated(0)
        del count_sketch
        torch.cuda.empty_cache()
        mem_after = torch.cuda.memory_allocated(0)
        print(f"Module memory: {(mem_before - mem_after)/1048576.0} MB")
    except NotImplementedError as e:
        print('Not Implemented')

ConutSketch:
Backend: Backends.PYTORCH
Module memory: 1.1796875 MB
Backend: Backends.PYTORCH_SPARSE
Module memory: 0.1796875 MB
Backend: Backends.PYG_SPARSE
Module memory: 0.1796875 MB
Backend: Backends.PYG_SCATTER
Module memory: 0.1953125 MB


### Backends

We test the sketching unit time with three types of inputs:

1. Sparse suqare matrix: 2048x2048, density=0.78%

```
ConutSketch:
Mode: dense_CPU, Backend: Backends.PYTORCH
Unit time: 11.4 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: dense_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 4.7 ms ± 420 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SPARSE
Unit time: 5.18 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SCATTER
Unit time: 18.2 ms ± 66.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH
Unit time: 166 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 589 µs ± 3.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SPARSE
Unit time: 120 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SCATTER
Unit time: 118 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: sparse_CPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 5.62 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SPARSE
Unit time: 1.16 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SCATTER
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 2.66 ms ± 57.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SPARSE
Unit time: 6.79 ms ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SCATTER
Not Implemented
```

2. Dense tall matrix: 2048x128, density=87.5%

```
ConutSketch:
Mode: dense_CPU, Backend: Backends.PYTORCH
Unit time: 884 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 557 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SPARSE
Unit time: 603 µs ± 8.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SCATTER
Unit time: 923 µs ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH
Unit time: 41.8 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 492 µs ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SPARSE
Unit time: 43.4 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SCATTER
Unit time: 45.3 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: sparse_CPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 19.5 ms ± 600 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SPARSE
Unit time: 1.15 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SCATTER
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 2.93 ms ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SPARSE
Unit time: 5.73 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SCATTER
Not Implemented
```

3. Sparse tall matrix: 2048x128, density=12.5%

```
ConutSketch:
Mode: dense_CPU, Backend: Backends.PYTORCH
Unit time: 880 µs ± 5.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 542 µs ± 8.19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SPARSE
Unit time: 607 µs ± 5.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_CPU, Backend: Backends.PYG_SCATTER
Unit time: 897 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH
Unit time: 42.1 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 497 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SPARSE
Unit time: 43.3 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: dense_GPU, Backend: Backends.PYG_SCATTER
Unit time: 45.1 µs ± 92.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Mode: sparse_CPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_CPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 3.85 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SPARSE
Unit time: 569 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Mode: sparse_CPU, Backend: Backends.PYG_SCATTER
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH
Not Implemented
Mode: sparse_GPU, Backend: Backends.PYTORCH_SPARSE
Unit time: 2.56 ms ± 72 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SPARSE
Unit time: 6.75 ms ± 73.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYG_SCATTER
Not Implemented
```

The conclusion on backend selection is the same, which is:

```
DEFAULT_COUNT_SKETCH_CONFIG = SketchConfig(
    dense_CPU=Backends.PYTORCH_SPARSE,
    dense_GPU=Backends.PYG_SPARSE,
    sparse_CPU=Backends.PYG_SPARSE,
    sparse_GPU=Backends.PYTORCH_SPARSE
)
```

### Sparse or Dense Output

Comparing the last two bulk timing results. We find that if memory allows, using dense representation is always faster. However, if the output is indeed sparse (density smaller than a threshold), we should keep the sparse representation even if it cost more time in preprocessing. Another option is to always use dense representation output, and convert to sparse representation if at the end if it is indeed sparse.

For simplicity, we can stick to use dense representation output here, no matter whether the final sketched matrix will be indeed sparse or not. This will also greatly reduce the number of possible backends of TensorSketch at this time. Later, we can further optimize this part by discussing when the intermediate output is sparse.

## Validate and Benchmark Tensor-Sketch

### Input

For TensorSketch, we only care the case when the input is a large square sparse matrix. We can use 2048x2048, density=0.78% as a test case.

### Backends

Two types of backends are considered: FFT-based (which reply on the implementation of CountSketch) or TensorSketch-based. Theoretically FFT-based is faster for degree>=2, however practically there is a chance that they are still comparable when degree<=3.

FFT-based backends are of high priority. We can use FFT even for sparse inputs, only need to convert to dense matrix before FFT and IFFT.

In [6]:
modes = ['sparse_CPU', 'sparse_GPU']
backends = [Backends.PYTORCH_FFT]

in_dim = 2048
out_dim = 512
feature_dim = in_dim
repeats = 128
log_interval = 32
ntrials = 3
max_degree = 3
avg_node_degree = 16

row = torch.arange(in_dim, dtype=torch.int64).repeat_interleave(avg_node_degree)
u0_col = torch.from_numpy(np.concatenate([np.random.choice(feature_dim, avg_node_degree, replace=False) for _ in range(in_dim)], axis=0, dtype=np.int64))
u0_val = torch.rand(avg_node_degree*in_dim, dtype=torch.float32)
u0_s = SparseTensor(row=row, col=u0_col, value=u0_val, sparse_sizes=(in_dim, feature_dim), is_sorted=False)
v0_col = torch.from_numpy(np.concatenate([np.random.choice(feature_dim, avg_node_degree, replace=False) for _ in range(in_dim)], axis=0, dtype=np.int64))
v0_val = torch.rand(avg_node_degree*in_dim, dtype=torch.float32)
v0_s = SparseTensor(row=row, col=v0_col, value=v0_val, sparse_sizes=(in_dim, feature_dim), is_sorted=False)
u0_d = u0_s.to_dense()
v0_d = v0_s.to_dense()

In [7]:
def validate_tensor_sketch(x0, y0, degree, config):
    device = x0.device if isinstance(x0, torch.Tensor) else x0.device()
    _prod = x0.t() @ y0
    prod = _prod if isinstance(_prod, torch.Tensor) else _prod.to_dense()
    pow_prod = torch.pow(prod, degree)
    rel_errors = np.zeros((repeats//log_interval, ntrials))
    density = 0
    for t in range(ntrials):
        app_pow_prod = torch.zeros(feature_dim, feature_dim).to(device)
        for r in range(1, repeats+1):
            tensor_sketch = initialize_single_layer_sketch_modules(in_dim, out_dim, degree, tensor_sketch_config=config)[1].to(device)
            _tsx0 = tensor_sketch(x0)[-1]
            _tsy0 = tensor_sketch(y0)[-1]
            app_pow_prod += _tsx0.T @ _tsy0
            tsx0 = _tsx0 if isinstance(_tsx0, torch.Tensor) else _tsx0.to_dense()
            density += torch.count_nonzero(tsx0)/out_dim/feature_dim
            if r % log_interval == 0:
                rel_errors[r//log_interval-1, t] = (torch.norm(pow_prod-app_pow_prod/r)/torch.norm(pow_prod)).cpu().numpy()
    for i in range(repeats//log_interval):
        avg_rel_error = np.mean(rel_errors, axis=1)
        std_rel_error = np.std(rel_errors, axis=1)
        print(f"# of repeats: {log_interval*(i+1):03d},\
              relative error: {avg_rel_error[i]*100:.2f}+/-{std_rel_error[i]*100:.2f}%")
    print(f"output density: {density/ntrials/repeats*100:.2f}%")


for degree in range(2, max_degree+1):
    print(f"TensorSketch: degree = {degree}")
    for mode in modes:
        for backend in backends:
            config = copy.copy(DEFAULT_TENSOR_SKETCH_CONFIG)
            setattr(config, mode, backend)
            print(f"Mode: {mode}, Backend: {backend}")
            if mode.startswith('dense'):
                x0, y0 = u0_d, v0_d
            else:
                x0, y0 = u0_s, v0_s
            if mode.endswith('CPU'):
                x0, y0 = x0.cpu(), y0.cpu()
            else:
                x0, y0 = x0.cuda(), y0.cuda()
            #try:
            validate_tensor_sketch(x0, y0, degree, config)
            #except NotImplementedError as e:
            #    print('Not Implemented')

TensorSketch: degree = 2
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
# of repeats: 032,              relative error: 301.85+/-0.11%
# of repeats: 064,              relative error: 213.53+/-0.09%
# of repeats: 096,              relative error: 174.38+/-0.06%
# of repeats: 128,              relative error: 150.99+/-0.06%
output density: 96.33%
Mode: sparse_GPU, Backend: Backends.PYTORCH_FFT
# of repeats: 032,              relative error: 301.93+/-0.03%
# of repeats: 064,              relative error: 213.44+/-0.04%
# of repeats: 096,              relative error: 174.24+/-0.13%
# of repeats: 128,              relative error: 150.90+/-0.09%
output density: 97.60%
TensorSketch: degree = 3
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
# of repeats: 032,              relative error: 2193.01+/-1.03%
# of repeats: 064,              relative error: 1549.76+/-0.82%
# of repeats: 096,              relative error: 1265.26+/-1.08%
# of repeats: 128,              relative error: 1096.36+/-0.76%


In [8]:
for degree in range(2, max_degree+1):
    print(f"TensorSketch: degree = {degree}")
    for mode in modes:
        for backend in backends:
            config = copy.copy(DEFAULT_TENSOR_SKETCH_CONFIG)
            setattr(config, mode, backend)
            print(f"Mode: {mode}, Backend: {backend}")
            if mode.startswith('dense'):
                x0, y0 = u0_d, v0_d
            else:
                x0, y0 = u0_s, v0_s
            if mode.endswith('CPU'):
                x0, y0 = x0.cpu(), y0.cpu()
                tensor_sketch = initialize_single_layer_sketch_modules(in_dim, out_dim, degree)[1].cpu()
            else:
                x0, y0 = x0.cuda(), y0.cuda()
                tensor_sketch = initialize_single_layer_sketch_modules(in_dim, out_dim, degree)[1].cuda()
            try:
                tensor_sketch(x0)
                print("Unit time: ", end="")
                %timeit tensor_sketch(x0)
            except NotImplementedError as e:
                print('Not Implemented')

TensorSketch: degree = 2
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
Unit time: 18.6 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Mode: sparse_GPU, Backend: Backends.PYTORCH_FFT
Unit time: 8.46 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
TensorSketch: degree = 3
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
Unit time: 24.3 ms ± 763 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Mode: sparse_GPU, Backend: Backends.PYTORCH_FFT
Unit time: 9.01 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Errors

Although we can see that the count sketch errors are large under this setup especially for degree=3, this is not an implementation mistake. The errors are large because you are sketching large square sparse matrices and you are comparing the approximated element-wise powers. This error metric is conceptually different to CountSketch's about which involves unsketching. In practice, we will not rely on this error bound explicitly.

### Timing

```
TensorSketch: degree = 2
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
Unit time: 12.1 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYTORCH_FFT
Unit time: 5.25 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
TensorSketch: degree = 3
Mode: sparse_CPU, Backend: Backends.PYTORCH_FFT
Unit time: 19.1 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Mode: sparse_GPU, Backend: Backends.PYTORCH_FFT
Unit time: 7.76 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
```


We see TensorSketch of degree=2 and 3 take up around 31.2ms/1.12ms=27.9x time than CountSketch of square sparse matrix of the same size and density. Thus, the real time bottleneck of sketching is on TensorSketch. There is a hope to optimize the process further by SFFT or unrolled TensorSketch for degree=2. But we may leave this to future.